<a href="https://colab.research.google.com/github/Agentoma/mmai5400A_assignment3/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipywidgets --upgrade
!pip install jupyter notebook --upgrade
!jupyter nbextension enable --py widgetsnbextension
!pip install "setfit[absa]"
!pip install spacy
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U
!spacy download en_core_web_lg
!spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.1 MB/s eta 0:00:00


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
import pandas as pd
from setfit import AbsaModel
import spacy

# Load spaCy model for aspect extraction
nlp = spacy.load("en_core_web_lg")

def absa(reviews):
    data = []
    try:
        # Load the fine-tuned Absa model
        model = AbsaModel.from_pretrained("models/setfit-absa-model")
    except Exception as e:
        print(f"Error loading the model: {e}")
        return pd.DataFrame(columns=["review_id", "dish", "sentiment"])

    # Define food-related keywords
    food_keywords = ["food", "dish", "meal", "taco", "rice", "meatball", "curry"]

    for idx, review in enumerate(reviews):
        try:
            # Process the review text
            doc = nlp(review)

            # Extract entities labeled as "PRODUCT" or containing food-related keywords
            dishes = [ent.text for ent in doc.ents if ent.label_ == "PRODUCT" or any(keyword in ent.text.lower() for keyword in food_keywords)]

            if not dishes:
                # If no dishes are found, skip this review
                continue

            # Get sentiment predictions for each dish
            for dish in dishes:
                # Extract the part of the review related to the dish
                aspect_sentence = " ".join([sent.text for sent in doc.sents if dish in sent.text])
                if aspect_sentence:
                    # Perform sentiment analysis
                    sentiment = model.predict([aspect_sentence])[0].capitalize()
                    data.append({
                        "review_id": idx,
                        "dish": dish,
                        "sentiment": sentiment
                    })
        except Exception as e:
            print(f"Error processing review {idx}: {e}")
            continue

    # Create and return the DataFrame
    return pd.DataFrame(data)
    print(pd.DataFrame(data))


Support for third party widgets will remain active for the duration of the session. To disable support:

In [4]:
from google.colab import output
output.disable_custom_widget_manager()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# New Section

In [5]:
import warnings
from tqdm import TqdmWarning
warnings.filterwarnings('ignore', category=TqdmWarning)

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, AutoTokenizer
from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments

# Load the fine-tuned model

In [6]:
!pip install "setfit[absa]"
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
# finetuning.py

from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback

def fine_tune_absa_model():
    # Load the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    train_dataset = dataset.select(range(128))  # Adjust as needed
    eval_dataset = dataset.select(range(128, 256))  # Adjust as needed

    # Initialize the model
    model = AbsaModel.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2",
        "sentence-transformers/all-mpnet-base-v2",
        spacy_model="en_core_web_lg",
    )

    # Set training arguments
    args = TrainingArguments(
        output_dir="models",
        num_epochs=5,
        use_amp=True,
        batch_size=128,
        evaluation_strategy="steps",
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Initialize the trainer
    trainer = AbsaTrainer(
        model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    # Train the model
    trainer.train()

    # Save the model
    model.save_pretrained("models/setfit-absa-model")

if __name__ == "__main__":
    fine_tune_absa_model()

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/147k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/46.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3693 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1134 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
The `evaluation_strategy` argument is deprecated and will be removed in a future version. Please use `eval_strategy` instead.


AttributeError: 'CallbackHandler' object has no attribute 'tokenizer'

In [ ]:
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    spacy_model="en_core_web_sm"
)
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    spacy_model="en_core_web_sm"
)

In [ ]:
# Third cell: Training setup and execution
# Create sample data - replace with your actual data
train_data = pd.DataFrame({
    'text': ['The food was great', 'Service was poor'],
    'span': ['food', 'service'],
    'polarity': [1, -1]
})

# Convert DataFrame to Dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)

# Define training arguments with correct parameters
args = TrainingArguments(
    output_dir="models",
    fp16=True,  # Replace use_amp with fp16
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Initialize trainer
trainer = AbsaTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset  # Replace with your eval data
)

# Train
trainer.train()

# Evaluate
metrics = trainer.evaluate()
print(metrics)

In [ ]:
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")


In [ ]:
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))

In [ ]:
args = TrainingArguments(
    output_dir="models",
    num_epochs=5,
    use_amp=True,
    batch_size=128,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)


In [ ]:
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from setfit import AbsaModel

# Download from the 🤗 Hub
model = AbsaModel.from_pretrained(
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect",
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity",
)
# Run inference
preds = model("The food was great, but the venue is just way too busy.")


# Load the model
model = AbsaModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Load dataset
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Define training arguments
args = TrainingArguments(
    output_dir="models",
    use_amp=True,
    per_device_train_batch_size=32,  # Adjust batch size as needed
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Initialize trainer
trainer = AbsaTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()

# Evaluate
metrics = trainer.evaluate(eval_dataset)
print(metrics)

# Save to Hub
trainer.push_to_hub("tomaarsen/setfit-absa-restaurants")


In [ ]:


splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/tomaarsen/setfit-absa-semeval-restaurants/" + splits["train"])
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    use_amp=True,
    per_device_train_batch_size=128,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)

In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Collecting setfit[absa]\n",
      "  Using cached setfit-1.1.0-py3-none-any.whl.metadata (12 kB)\n",
      "Collecting datasets>=2.15.0 (from setfit[absa])\n",
      "  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)\n",
      "Collecting sentence-transformers>=3 (from sentence-transformers[train]>=3->setfit[absa])\n",
      "  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)\n",
      "[notice] A new release of pip is available: 24.2 -> 24.3.1\n",
      "[notice] To update, run: python.exe -m pip install --upgrade pip\n"
     ]
    }
   ],
   "source": [
    "!pip install setfit[absa]",
    "!pip install spacy",
    "!pip install en-core-web-lg==3.7.1",
    "!python -m spacy download en_core_web_lg"
   ]
  }
 ]
}

In [ ]:
# Install necessary libraries
!pip install "setfit[absa]" spacy
!spacy download en_core_web_sm

# Ensure compatibility
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U

from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset


In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect")
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants")
tokenized_datasets = dataset.map(lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length"), batched=True)

In [ ]:

# Initialize the AbsaModel
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    spacy_model="en_core_web_sm",
)

# Load the dataset
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))


In [ ]:

# Set up training arguments
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    fp16=True
)

# Initialize and train the AbsaTrainer
trainer = AbsaTrainer(
    model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

# Save the trained model
trainer.save_model("absa_model")

In [ ]:
# Install necessary libraries
!pip install "setfit[absa]" spacy
!spacy download en_core_web_sm

# Ensure compatibility
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U

from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# Set up training arguments
# The original code attempts to use TrainingArguments.copy() which does not exist.
# We now create a new TrainingArguments object for polarity_args using the values of args.
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    fp16=True
)

# Initialize and train the AbsaTrainer
# Pass args as polarity_args so that a new TrainingArguments object is created internally.
trainer = AbsaTrainer(
    model,
    args=args,
    polarity_args=args,  # Pass args to polarity_args
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

# Save the trained model
trainer.save_model("absa_model")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers import SentenceTransformer

# Load the training dataset
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants")

# Load pre-trained sentence embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_name)

# Initialize SetFit model
setfit_model = SetFitModel.from_pretrained(model_name)

# Set training arguments
trainer = SetFitTrainer(
    model=setfit_model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    batch_size=16,
    num_epochs=5,
    learning_rate=1e-3,
    seed=42
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained("models/setfit-absa-model")

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import load_dataset

def fine_tune_absa_model():
    # Load the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    train_dataset = dataset.select(range(128))  # Adjust as needed
    eval_dataset = dataset.select(range(128, 256))  # Adjust as needed

    # Initialize the model
    model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

    # Set training arguments
    args = TrainingArguments(
        output_dir="models",
        num_epochs=5,
        use_amp=True,
        batch_size=16,
        eval_strategy="steps",
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Initialize the trainer
    trainer = SetFitTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # Train the model
    trainer.train()

    # Save the model
    trainer.model.save_pretrained("models/setfit-absa-model")

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import load_dataset

def fine_tune_absa_model():
    # Load the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    train_dataset = dataset.select(range(110))  # Adjust as needed
    eval_dataset = dataset.select(range(110, 256))  # Adjust as needed

    # Initialize the model
    model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

    # Initialize the trainer with training arguments
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        num_epochs=5,
        batch_size=16,
        use_amp=True,
        eval_strategy="steps",
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Train the model
    trainer.train()

    # Save the model
    trainer.model.save_pretrained("models/setfit-absa-model")

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import load_dataset

def fine_tune_absa_model():
    # Load the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    train_dataset = dataset.select(range(128))  # Adjust as needed
    eval_dataset = dataset.select(range(128, 256))  # Adjust as needed

    # Initialize the model
    model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

    # Initialize the trainer with training arguments
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        num_epochs=5,
        batch_size=16,
        use_amp=True,
        evaluation_strategy="steps",
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Train the model
    trainer.train()

    # Save the model
    trainer.model.save_pretrained("models/setfit-absa-model")

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
!pip install --upgrade setfit transformers

In [ ]:
from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback

def fine_tune_absa_model():
    # Load the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    train_dataset = dataset.select(range(128))
    eval_dataset = dataset.select(range(128, 256))

    # Initialize the model
    model = AbsaModel.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2",
        "sentence-transformers/all-mpnet-base-v2",
        spacy_model="en_core_web_lg",
    )

    # Set training arguments
    args = TrainingArguments(
        output_dir="models",
        num_epochs=5,
        use_amp=True,
        batch_size=128,
        eval_strategy="steps",  # Use eval_strategy instead of evaluation_strategy
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Initialize the trainer
    trainer = AbsaTrainer(
        model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    # Train the model
    trainer.train()

    # Save the model
    model.save_pretrained("models/setfit-absa-model")

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
import pandas as pd
from setfit import SetFitModel
import spacy

# Load spaCy model for aspect extraction
nlp = spacy.load("en_core_web_lg")

def absa(reviews):
    try:
        # Load the fine-tuned SetFit model
        model = SetFitModel.from_pretrained("models/setfit-absa-model")
    except Exception as e:
        print(f"Error loading the model: {e}")
        return pd.DataFrame(columns=["review_id", "dish", "sentiment"])

    data = []
    for idx, review in enumerate(reviews):
        try:
            # Process the review text
            doc = nlp(review)

            # Define a list of keywords related to dishes
            food_keywords = ["food", "dish", "meal", "taco", "rice", "meatball", "curry"]

            # Extract entities labeled as "PRODUCT" or containing food-related keywords
            dishes = [ent.text for ent in doc.ents if ent.label_ == "PRODUCT" or any(keyword in ent.text.lower() for keyword in food_keywords)]

            if not dishes:
                # If no dishes are found, skip this review
                continue

            # Get sentiment predictions for each dish
            for dish in dishes:
                # Extract the part of the review related to the dish
                aspect_sentence = " ".join([sent.text for sent in doc.sents if dish in sent.text])
                if aspect_sentence:
                    # Perform sentiment analysis
                    sentiment = model.predict([aspect_sentence])[0].capitalize()
                    data.append({
                        "review_id": idx,
                        "dish": dish,
                        "sentiment": sentiment
                    })
        except Exception as e:
            print(f"Error processing review {idx}: {e}")
            continue

    # Create and return the DataFrame
    return pd.DataFrame(data)
    print(data.from_pretrained("models/setfit-absa-model"))

In [ ]:
from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback

def fine_tune_absa_model():
    # Load the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    train_dataset = dataset.select(range(128))
    eval_dataset = dataset.select(range(128, 256))

    # Initialize the model
    model = AbsaModel.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2",
        "sentence-transformers/all-mpnet-base-v2",
        spacy_model="en_core_web_lg",
    )

    # Set training arguments
    args = TrainingArguments(
        output_dir="models",
        num_epochs=5,
        use_amp=True,
        batch_size=128,
        eval_strategy="steps",  # Use eval_strategy instead of evaluation_strategy
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Initialize the trainer
    trainer = AbsaTrainer(
        model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    # Train the model
    trainer.train()

    # Save the model
    model.save_pretrained("models/setfit-absa-model")

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback

def fine_tune_absa_model():
    # Load the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    train_dataset = dataset.select(range(128))
    eval_dataset = dataset.select(range(128, 256))

    # Initialize the model
    model = AbsaModel.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2",
        "sentence-transformers/all-mpnet-base-v2",
        spacy_model="en_core_web_lg",
    )

    # Set training arguments
    args = TrainingArguments(
        output_dir="models",
        num_epochs=5,
        use_amp=True,
        batch_size=128,
        eval_strategy="steps",  # Use eval_strategy instead of evaluation_strategy
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Initialize the trainer
    trainer = AbsaTrainer(
        model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    # Train the model
    trainer.train()

    # Save the model
    model.save_pretrained("models/setfit-absa-model")

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
from setfit import AbsaModel, AbsaTrainer
from datasets import load_dataset
from transformers import TrainingArguments, EarlyStoppingCallback
import os

def fine_tune_absa_model():
    # Create output directory if it doesn't exist
    os.makedirs("models/setfit-absa-model", exist_ok=True)

    # Load and shuffle the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    dataset = dataset.rename_column("label", "polarity")
    dataset = dataset.shuffle(seed=42)

    # Split the dataset into train and evaluation datasets
    train_dataset = dataset.select(range(128))  # Adjust based on dataset size
    eval_dataset = dataset.select(range(128, 256))  # Adjust based on dataset size

    # Initialize the model
    model = AbsaModel.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2",
        spacy_model="en_core_web_sm",
    )

    # Initialize the training arguments with early stopping patience
    args = TrainingArguments(
        output_dir="models/setfit-absa-model",
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="steps",
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Initialize the trainer with early stopping callback
    trainer = AbsaTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    # Train the model
    trainer.train()

    # Save the model with separate directories for aspect and polarity models
    model.save_pretrained(
        "models/setfit-absa-model-aspect",
        "models/setfit-absa-model-polarity",
    )

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
from setfit import AbsaModel, AbsaTrainer
from datasets import load_dataset
from transformers import TrainingArguments, EarlyStoppingCallback
import os

def fine_tune_absa_model():
    # Create output directory if it doesn't exist
    os.makedirs("models/setfit-absa-model", exist_ok=True)

    # Load and shuffle the dataset
    dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
    # Rename the "label" column to "polarity" if it exists
    if "label" in dataset.features:
        dataset = dataset.rename_column("label", "polarity")
    dataset = dataset.shuffle(seed=42)

    # Split the dataset into train and evaluation datasets
    train_dataset = dataset.select(range(128))  # Adjust based on dataset size
    eval_dataset = dataset.select(range(128, 256))  # Adjust based on dataset size

    # Initialize the model
    model = AbsaModel.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2",
        spacy_model="en_core_web_sm",
    )

    # Initialize the training arguments with early stopping patience
    args = TrainingArguments(
        output_dir="models/setfit-absa-model",
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="steps",
        eval_steps=50,
        save_steps=50,
        load_best_model_at_end=True,
    )

    # Initialize the trainer with early stopping callback
    trainer = AbsaTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    # Train the model
    trainer.train()

    # Save the model with separate directories for aspect and polarity models
    model.save_pretrained(
        "models/setfit-absa-model-aspect",
        "models/setfit-absa-model-polarity",
    )

if __name__ == "__main__":
    fine_tune_absa_model()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()